In [20]:
import numpy as np
import cv2
from glob import glob
import matplotlib.pyplot as plt
import pickle
%matplotlib inline



In [23]:

def getPatch(img_orig, patchSize, r):
    if len(img_orig.shape) == 3:
        img = cv2.cvtColor(img_orig, cv2.COLOR_RGB2GRAY)
    else:
        img = img_orig

    #Choose random point as the center of the patch
    x = np.random.randint(patchSize//2+r, img.shape[1]-(patchSize//2+r))  
    y = np.random.randint(patchSize//2+r, img.shape[0]-(patchSize//2+r)) 
    center = (x, y)
    src = []
    #calculate 4 corner points of the patch
    for i in range(4):
        angle = i*np.pi/2
        x_offset = int(patchSize//2*np.cos(angle))
        y_offset = int(patchSize//2*np.sin(angle))
        src.append((x+x_offset, y+y_offset))

    dst = [(pt[0]+np.random.randint(-r, r), pt[1]+np.random.randint(-r, r)) for pt in src]
    
    H = cv2.getPerspectiveTransform(np.float32(src), np.float32(dst))
    H_inv = np.linalg.inv(H)
    warpImg = cv2.warpPerspective(img, H_inv, (img.shape[1],img.shape[0]))
    # Plotting Warped Image
#     tempImg = warpImg.copy()
#     cv2.polylines(tempImg, np.int32([src]), 1, (0),2)
#     plt.figure()
#     plt.imshow(tempImg, cmap='gray')
#     plt.show()
    
    patch1 = img[y-patchSize//2:y+patchSize//2, x-patchSize//2:x+patchSize//2]
    patch2 = warpImg[y-patchSize//2:y+patchSize//2, x-patchSize//2:x+patchSize//2]
    imgData = np.dstack((patch1, patch2))
    H4Pt = np.subtract(np.array(dst), np.array(src))
    return imgData, H4Pt # return the image and the homography matrix



def dataGen(Image,size=(640,480),patchSize=128,r=32):
    X=[]
    Y=[]
    for j in range(1):
        print("No. of samples collected = "+str(len(Image)*j))
        for i in range(len(Image)):  
            img = plt.imread(Image[i])
            img = cv2.resize(img,size)
            imgData,hData = getPatch(img,patchSize,r)
            X.append(imgData)
            Y.append(hData)
    return X,Y


trainImg = glob("../Data/Train/*.jpg")
valImg = glob("../Data/Val/*.jpg")
Basepath = "../Data/Train/"
print("No. of Training Images = " + str(len(trainImg)))
print("No. of Validation Images = " + str(len(valImg)))


X_train=[]
Y_train=[]
X_val = []
Y_val = []

X_train,Y_train = dataGen(trainImg)
X_val,Y_val = dataGen(valImg)

training = {'features': X_train, 'labels': Y_train}
pickle.dump(training, open("unsupervised.pkl", "wb"))
validation = {'features': X_val, 'labels': Y_val}
pickle.dump(validation, open("validation_less.pkl", "wb"))




No. of Training Images = 5000
No. of Validation Images = 1000
No. of samples collected = 0
No. of samples collected = 0


In [24]:
import pickle
import numpy as np

def extract(data):
    f = open(data, 'rb')
    out = pickle.load(f)
    features = np.array(out['features'])
    labels = np.array(out['Corners'])
    f.close()
    return features,labels


features,labels=extract('training.pkl')
print(labels[0])
t = labels[0].reshape((1,8))
print(t.shape)
print(t[0].shape)
print(features[0])

features,corners = extract('unsup.pkl')


print(features[0].shape)
print(corners[0].shape)

